In [33]:
import torch.nn as nn
import torch

In [34]:
import ast

input_dir="data"

with open(input_dir + "/mfcc/lang_feature_file.csv",'r') as f:
    ls = f.readlines()


In [35]:
import ast

l=ls[0]
r = l[0:-1].split(',',2)
x = r[0]
y = ast.literal_eval(r[2].strip())

In [36]:
import ast
import random

input_dir="data"

records = []

with open(input_dir + "/mfcc/lang_feature_file.csv", 'r') as f:
    for l in f:
        r = l[0:-1].split(',',2)
        records.append((r[0], r[1], ast.literal_eval(r[2].strip())))
random.shuffle(records)
print("records: %s" % len(records))

records: 400


In [37]:
langs = sorted(list(set([r[0] for r in records])))

In [38]:
lang_index = {l: i for i, l in enumerate(langs)}
index_lang = {i: l for i, l in lang_index}

In [39]:
t_n = 400 * 2 // 3
train=records[0:t_n]
valid=records[t_n:]

In [60]:
x_train_o = [r[2][:100] for r in train]
y_train_o = [lang_index[r[0]] for r in train]

In [41]:
x_valid_o = [r[2][:100] for r in valid]
y_valid_o = [lang_index[r[0]] for r in valid]

In [42]:
x_train_o, y_train_o, x_valid_o, y_valid_o = map(
    torch.tensor, (x_train_o, y_train_o, x_valid_o, y_valid_o)
)

In [43]:
x_train = x_train_o.reshape(x_train_o.shape[0],-1)
x_valid = x_valid_o.reshape(x_valid_o.shape[0],-1)

In [61]:
y_train = y_train_o
y_valid = y_valid_o

In [44]:
x_train.shape

torch.Size([266, 1300])

In [45]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [46]:
import torch.nn.functional as F
loss_func = F.cross_entropy

In [47]:
input_size = x_train.shape[1]
output_size = len(langs)

In [90]:
bs=64
xb=x_train[0:bs]
xb=xb.view(xb.shape[0],1,100 * 13)
conv1 = nn.Conv1d(1, 20, kernel_size=13, stride=1, padding=0)
xb = conv1(xb)
xb=xb.view(xb.shape[0],1,20,1288)
xb.shape
conv2 = nn.Conv2d(1, 1, kernel_size=(5,5), stride=1, padding=0)
xb=conv2(xb)
m = nn.MaxPool2d(3, stride=2)
xb = m(xb)
#lin1 = nn.Linear(16 * 1284, 30)
#xb = xb.view(bs, -1)
#xb=lin1(xb)
xb.shape


torch.Size([64, 1, 7, 641])

In [91]:
class Mnist_CNN_(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 20, kernel_size=13, stride=1, padding=0)        
        self.d1 = nn.Dropout(0.7)
        self.conv2 = nn.Conv2d(1, 1, kernel_size=(5,5), stride=1, padding=0)
        self.d2 = nn.Dropout(0.7)
        self.m = nn.MaxPool2d(3, stride=2)
        self.lin1 = nn.Linear(7* 641, 30)
        self.d5 = nn.Dropout(0.6)
        self.lin2 = nn.Linear(150, 30)
        self.d6 = nn.Dropout(0.6)
        self.lin3 = nn.Linear(30, 10)
        self.d7 = nn.Dropout(0.8)
        self.lin4 = nn.Linear(30, output_size)
        self.d8 = nn.Dropout(0.8)
    def forward(self, xb):
        bs = xb.shape[0]
        xb=xb.view(xb.shape[0],1,100 * 13)
        xb = self.d1(self.conv1(xb))
        xb=xb.view(xb.shape[0],1,20,1288)
        xb = self.d2(self.conv2(xb))
        xb = self.m(xb)
        #xb = self.d3(self.conv3(xb))
        #xb = self.d4(self.conv4(xb))
        xb = xb.view(bs, -1)
        xb = self.d5(F.relu(self.lin1(xb)))
        #xb = self.d6(F.relu(self.lin2(xb)))
        #xb = self.d7(F.relu(self.lin3(xb)))
        xb = self.lin4(xb)
        return xb

In [149]:
s=100
f=13
c1=5
bs=64
xb=x_train[0:bs]
xb=xb.view(xb.shape[0],1,100,13)
conv1 = nn.Conv2d(1, 5, kernel_size=(2,2), stride=1, padding=0)
xb1 = conv1(xb)
print(xb1.shape)
#xb1 = F.avg_pool2d(xb1, (2, 1))
#print(xb1.shape)
conv2 = nn.Conv2d(5, 5, kernel_size=(2,2), stride=2, padding=0, dilation=2)
xb2 = conv2(xb1)
print(xb2.shape)
conv3 = nn.Conv2d(5, 5, kernel_size=(2,2), stride=4, padding=0, dilation=4)
xb3 = conv3(xb2)
print(xb3.shape)
#conv4 = nn.Conv2d(1, 1, kernel_size=(10,1), stride=1, padding=0, dilation=1)
#xb4 = conv4(xb3)
#print(xb4.shape)


torch.Size([64, 5, 99, 12])
torch.Size([64, 5, 49, 5])
torch.Size([64, 5, 12, 1])


In [155]:
class Mnist_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 5, kernel_size=(2,2), stride=1, padding=0)
        self.d1 = nn.Dropout(0.7)
        self.conv2 = nn.Conv2d(5, 5, kernel_size=(2,2), stride=2, padding=0, dilation=2)
        self.d2 = nn.Dropout(0.7)
        self.conv3 = nn.Conv2d(5, 5, kernel_size=(2,2), stride=4, padding=0, dilation=4)
        self.d3 = nn.Dropout(0.6)
        self.conv4 = nn.Conv2d(1, 1, kernel_size=(10,1), stride=(1,1), padding=0, dilation=1)
        self.d4 = nn.Dropout(0.6)
        self.lin1 = nn.Linear(5 * 12 * 1, 30)
        self.d5 = nn.Dropout(0.6)
        self.lin2 = nn.Linear(150, 30)
        self.d6 = nn.Dropout(0.6)
        self.lin3 = nn.Linear(30, 10)
        self.d7 = nn.Dropout(0.8)
        self.lin4 = nn.Linear(30, output_size)
        self.d8 = nn.Dropout(0.8)
    def forward(self, xb):
        bs = xb.shape[0]
        xb = xb.view(-1, 1, 100, 13)
        xb = self.d1(self.conv1(xb))
        xb = self.d2(self.conv2(xb))
        xb = self.d3(self.conv3(xb))
        #xb = self.d4(self.conv4(xb))
        xb = xb.view(bs, -1)
        xb = self.d5(F.relu(self.lin1(xb)))
        #xb = self.d6(F.relu(self.lin2(xb)))
        #xb = self.d7(F.relu(self.lin3(xb)))
        xb = self.lin4(xb)
        return xb

In [156]:
class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(input_size, output_size)

    def forward(self, xb):
        return self.lin(xb)

In [157]:
class Mnist_MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.k1=4
        self.c1=5
        self.conv1 = nn.Conv2d(1, self.c1, kernel_size=(5,self.k1), stride=1, padding=0)
        self.d0 = nn.Dropout(0.6)
        self.conv2 = nn.Conv2d(self.c1, 1, kernel_size=(9,5), stride=1, padding=1)
        self.d01 = nn.Dropout(0.6)
        self.lin1 = nn.Linear(285, 50)
        self.d1 = nn.Dropout(0.6)
        self.lin2 = nn.Linear(50, 30)
        self.d2 = nn.Dropout(0.6)
        self.lin3 = nn.Linear(30, output_size)
        #self.d3 = nn.Dropout(0.6)
        #self.lin4 = nn.Linear(20, output_size)

    def forward(self, xb):
        bs=xb.shape[0]
        xb = xb.view(-1, 1, 13, 100)
        xb = self.conv1(xb)
        xb = self.d0(xb)
        #print(xb.shape)
        #print(xb.shape)
        xb = self.conv2(xb)
        #print(xb.shape)
        #xb = F.avg_pool2d(xb, (self.c1, 1))

        #print(input_size)
        #print(self.c1 * (input_size//13 - self.k1 + 1))
        xb = xb.view(bs, -1)
        #print(xb.shape)
        xb = F.relu(self.lin1(xb))
        #xb = self.d1(xb)
        #xb = F.relu(self.lin2(xb))
        #xb = self.d2(xb)
        #xb = self.lin3(xb)
        return xb

In [158]:
from torch import optim

lr = 0.0005 # 0.0001 # 
def get_model():
    model = Mnist_CNN() #Mnist_MLP() #Mnist_Logistic() #Mnist_MLP()
    return model, optim.Adam(model.parameters(), lr=lr)


In [159]:
model, opt = get_model()

n = x_train.shape[0]
bs = 64  # batch size

epochs = 500  # how many epochs to train for
best_valid_acc = 0
for epoch in range(epochs):
    model.train()
    for i in range((n - 1) // bs + 1): # for all minibatches (n number of training datapoints)
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()
    
    model.eval()
    with torch.no_grad():
        valid_acc = accuracy(model(x_valid), y_valid)
        if valid_acc > best_valid_acc:
            best_valid_acc = valid_acc
            print("valid acc increased to %s" % valid_acc) 
        else:
            if  valid_acc / best_valid_acc < 0.7:
                print("valid acc deccreased to %s" % valid_acc) 
                #break
    #print("epoch: %s" % epoch)
print("best valid acc %s" % best_valid_acc)
model.eval()
train_acc = accuracy(model(x_train), y_train)
print("train acc %s" % train_acc) 


valid acc increased to tensor(0.2985)
valid acc increased to tensor(0.3060)
valid acc increased to tensor(0.3134)
best valid acc tensor(0.3134)
train acc tensor(0.2744)
